In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

## 2024 데이터 전처리

In [2]:
df = pd.read_excel('data/2024/2024 외래관광객조사_Data.xlsx')

- 국적: D_NAT
- 체류기간: M일HAP
- 동반자 유무: Q7A
- 동행자 유형: Q7a_dk ~ Q7a8
- 여행 형태: D_GUB
- 방문 목적: D_MOK
- 총지출: 총액1인TOT항공제외2
- 참여한 활동(자연경관 감상, 뷰티/미용 관광): Q8a03, Q8a12
- 참여 만족도: Q8_1a1, Q8_1a2, Q8_1a3
- 가중치: weight 

In [32]:
cols_needed = ['D_NAT','M일HAP', 
               'Q7A','Q7a_dk', 'Q7a2', 'Q7a3', 'Q7a4', 'Q7a5', 'Q7a6', 'Q7a7', 'Q7a8',
               'D_GUB', 'D_MOK', '총액1인TOT_개별국제교통비제외2', 
               'Q8a03', 'Q8a12', 
               'Q8_1a1', 'Q8_1a2', 'Q8_1a3',
               'D_AGE']
df = df[cols_needed]

In [33]:
df.isnull().sum()

D_NAT                    0
M일HAP                    0
Q7A                      0
Q7a_dk                6934
Q7a2                  6655
Q7a3                  8024
Q7a4                  7720
Q7a5                  7730
Q7a6                  5687
Q7a7                  8309
Q7a8                  8734
D_GUB                    0
D_MOK                    0
총액1인TOT_개별국제교통비제외2       0
Q8a03                 3560
Q8a12                 7459
Q8_1a1                   0
Q8_1a2                 174
Q8_1a3                1142
D_AGE                    0
dtype: int64

### 동행자 유형 분류

In [34]:
# 1. 동행자 관련 컬럼만 추출
companion_cols = ['Q7a_dk', 'Q7a2', 'Q7a3', 'Q7a4', 'Q7a5', 'Q7a6', 'Q7a7', 'Q7a8']

# 2. 결측치 처리 안함 
# 결측치는 <선택하지 않음>을 의미하므로 처리하지 않음

# 3. 동행자유형 파생 변수 만들기
## 1) 단순 그룹 (가족 여부 중심)
def classify_family_group(row):
    if row['Q7a_dk'] == 1:
        return '혼자'
    if any([row['Q7a2'] == 2, row['Q7a3'] == 3, row['Q7a4'] == 4, row['Q7a5'] == 5]):
        return '가족'
    if row['Q7a6'] == 6:
        return '친구'
    if row['Q7a7'] == 7:
        return '직장동료'
    if row['Q7a8'] == 8:
        return '기타'
    return '응답없음'

## 2) 세분화 그룹 (구체 동행자 중심)
def classify_detailed_group(row):
    if row['Q7a_dk'] == 1:
        return '혼자'
    if row['Q7a2'] == 2:
        return '배우자/파트너'
    if row['Q7a3'] == 3:
        return '부모님'
    if row['Q7a4'] == 4:
        return '자녀'
    if row['Q7a5'] == 5:
        return '그 외 가족 친지'
    if row['Q7a6'] == 6:
        return '친구'
    if row['Q7a7'] == 7:
        return '직장동료'
    if row['Q7a8'] == 8:
        return '기타'
    return '응답없음'

# 4. 적용
df['동행자_단순'] = df.apply(classify_family_group, axis=1)
df['동행자_세분화'] = df.apply(classify_detailed_group, axis=1)

# 5. 유효 응답만 남기기
## 두 변수 모두 '응답없음'이 아닌 행만 남김
df = df[(df['동행자_단순'] != '응답없음') & (df['동행자_세분화'] != '응답없음')].copy()

C:\Users\82106\AppData\Local\Temp\ipykernel_17208\2849500916.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['동행자_단순'] = df.apply(classify_family_group, axis=1)
C:\Users\82106\AppData\Local\Temp\ipykernel_17208\2849500916.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['동행자_세분화'] = df.apply(classify_detailed_group, axis=1)


In [35]:
df.columns

Index(['D_NAT', 'M일HAP', 'Q7A', 'Q7a_dk', 'Q7a2', 'Q7a3', 'Q7a4', 'Q7a5',
       'Q7a6', 'Q7a7', 'Q7a8', 'D_GUB', 'D_MOK', '총액1인TOT_개별국제교통비제외2', 'Q8a03',
       'Q8a12', 'Q8_1a1', 'Q8_1a2', 'Q8_1a3', 'D_AGE', '동행자_단순', '동행자_세분화'],
      dtype='object')

In [36]:
df['동행자_단순'].value_counts()

동행자_단순
가족      3704
친구      2854
혼자      1835
직장동료     355
기타        21
Name: count, dtype: int64

In [37]:
df['동행자_세분화'].value_counts()

동행자_세분화
친구           2854
배우자/파트너      2114
혼자           1835
부모님           630
그 외 가족 친지     527
자녀            433
직장동료          355
기타             21
Name: count, dtype: int64

### 국적 선별 
- 2023년 글로벌 방한 관광객 통계 기준 상위 6개국 
- GWI-Rankings 상위 5개국 중 2개국(방한관광객 상위 국가와 중복 제외)

In [38]:
df['D_NAT'].value_counts()

D_NAT
1     1804
2     1716
3     1171
4      786
5      746
7      603
10     537
6      515
12     327
18     282
16     282
Name: count, dtype: int64

### 국가명 매핑

In [39]:
df['D_NAT'].dtype

dtype('int64')

In [40]:
nat_map = {
    1: '중국',
    2: '일본',
    3: '대만',
    4: '미국',
    5: '홍콩',
    6: '태국',
    7: '베트남',
    10: '싱가포르',
    16: '영국',
    18: '독일',
    12: '중동'
}

df['국가명'] = df['D_NAT'].map(nat_map)

# 분석 대상 국가만 필터링
df = df[df['국가명'].notna()]

In [41]:
df['국가명'].value_counts()

국가명
중국      1804
일본      1716
대만      1171
미국       786
홍콩       746
베트남      603
싱가포르     537
태국       515
중동       327
독일       282
영국       282
Name: count, dtype: int64

### 연령대 분류

In [42]:
# 연령 통합 그룹
def classify_age_group_simple(code):
    if code == 1:
        return '10대'
    elif code == 2:
        return '20대'
    elif code in [3, 4, 5]:
        return '30~50대'
    elif code == 6:
        return '60대 이상'
    return '응답없음'

# 연령 세분화 그룹 (항목 정의서 그대로)
def classify_age_group_detailed(code):
    age_map = {
        1: '10대',
        2: '20대',
        3: '30대',
        4: '40대',
        5: '50대',
        6: '60대 이상'
    }
    return age_map.get(code, '응답없음')

# 적용
df['연령대_단순'] = df['D_AGE'].apply(classify_age_group_simple)
df['연령대_세분화'] = df['D_AGE'].apply(classify_age_group_detailed)

df = df[(df['연령대_단순'] != '응답없음') & (df['연령대_세분화'] != '응답없음')].copy()

In [43]:
df

,D_NAT,M일HAP,Q7A,Q7a_dk,Q7a2,Q7a3,Q7a4,Q7a5,Q7a6,Q7a7,...,Q8a12,Q8_1a1,Q8_1a2,Q8_1a3,D_AGE,동행자_단순,동행자_세분화,국가명,연령대_단순,연령대_세분화
1,3,5,2,NaN,2.0,NaN,4.0,NaN,NaN,NaN,...,NaN,1,2.0,5.0,4,가족,배우자/파트너,대만,30~50대,40대
2,3,6,2,NaN,NaN,NaN,4.0,NaN,NaN,7.0,...,12.0,12,1.0,2.0,3,가족,자녀,대만,30~50대,30대
3,1,6,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,11,1.0,3.0,2,혼자,혼자,중국,20대,20대
4,3,5,2,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,12.0,2,12.0,5.0,2,가족,부모님,대만,20대,20대
5,3,5,2,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,12.0,12,6.0,1.0,2,가족,부모님,대만,20대,20대
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16211,1,5,2,NaN,NaN,NaN,NaN,NaN,6.0,NaN,...,NaN,3,2.0,1.0,4,친구,친구,중국,30~50대,40대
16212,1,5,2,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,2,1.0,NaN,4,가족,배우자/파트너,중국,30~50대,40대
16213,1,5,2,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,13,3.0,2.0,5,가족,자녀,중국,30~50대,50대
16214,1,5,2,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,3,2.0,NaN,5,가족,배우자/파트너,중국,30~50대,50대


### 체류일수 및 총지출 0 이하 제거

In [44]:
df = df[(df['M일HAP'] > 0) & (df['총액1인TOT_개별국제교통비제외2'] > 0)]

### 웰니스 활동 참여 여부 파생변수 생성

In [45]:
df['Q8a03'].dtype

dtype('float64')

In [46]:
df['Q8a12'].dtype

dtype('float64')

In [47]:
df['웰니스참여'] = ((df['Q8a03'] == 3) | (df['Q8a12'] == 12)).astype(int)

In [48]:
df['웰니스참여'].value_counts()

웰니스참여
1    5690
0    3079
Name: count, dtype: int64

In [49]:
df['Q8_1a1'].value_counts()

Q8_1a1
2     1886
1     1754
3     1329
5      932
8      505
13     398
12     341
6      332
7      250
11     237
4      233
14     220
9      106
16      83
10      56
20      56
15      33
18       9
17       5
19       4
Name: count, dtype: int64

In [50]:
df['Q8_1a2'].value_counts()

Q8_1a2
1.0     2625
2.0     2214
3.0     1083
5.0      795
6.0      309
7.0      281
4.0      271
12.0     226
13.0     187
11.0     172
8.0      151
9.0       79
14.0      71
16.0      48
10.0      38
15.0      29
20.0      10
18.0       5
17.0       1
Name: count, dtype: int64

In [51]:
df['Q8_1a3'].value_counts()

Q8_1a3
2.0     1725
1.0     1627
3.0     1178
5.0      856
7.0      356
4.0      326
6.0      310
13.0     293
12.0     242
11.0     225
8.0      173
14.0     113
10.0      56
9.0       52
16.0      36
15.0      29
20.0      16
19.0       6
18.0       4
17.0       4
Name: count, dtype: int64

- 자연경관(3)은 Top3 만족 활동 중 매우 자주 등장 (총 4000회 이상)

- 뷰티/미용(12)도 상대적으로 빈도는 적지만 분명히 선택되고 있음

### 웰니스 활동이 Top3 만족 활동에 포함된 총 응답자 수
- 현재는 각 코드별 총 등장 수만 계산됨 -> 실제 몇 명이 ‘3 또는 12’를 한번 이상 선택했는지 계산

In [52]:
df['웰니스만족'] = df[['Q8_1a1', 'Q8_1a2', 'Q8_1a3']].apply(
    lambda row: any(x in [3, 12] for x in row if pd.notnull(x)),
    axis=1
).astype(int)

df['웰니스만족'].value_counts()

웰니스만족
0    4551
1    4218
Name: count, dtype: int64

### 여행 목적 컬럼 (D_MOK) 전처리
- 여가/위락/힐링, 친구/친지 방문, 사업/전문 활동, 교육, 기타
- 여가/힐링을 즐길 의사와 여건이 있었던 응답자만 포함

In [53]:
df['D_MOK'].dtype

dtype('int64')

In [54]:
df = df[df['D_MOK'].isin([1, 2])]

In [55]:
df['D_MOK'].value_counts()

D_MOK
1    7374
2    1395
Name: count, dtype: int64

In [56]:
# 분류 성공 여부 및 결측 확인
# 결측치 확인
print("동행자_단순 결측치 수:", df['동행자_단순'].isnull().sum())

# 고유값 확인
print("동행자_단순 고유값 목록:", df['동행자_단순'].dropna().unique())

print("동행자_세분화 결측치 수:", df['동행자_세분화'].isnull().sum())
print("동행자_세분화 고유값 목록:", df['동행자_세분화'].dropna().unique())

동행자_단순 결측치 수: 0
동행자_단순 고유값 목록: ['가족' '혼자' '친구' '직장동료' '기타']
동행자_세분화 결측치 수: 0
동행자_세분화 고유값 목록: ['배우자/파트너' '자녀' '혼자' '부모님' '친구' '그 외 가족 친지' '직장동료' '기타']


In [58]:
# 분류 성공 여부 및 결측 확인
# 결측치 확인
print("연령대_단순 결측치 수:", df['연령대_단순'].isnull().sum())

# 고유값 확인
print("연령대_단순 고유값 목록:", df['연령대_단순'].dropna().unique())

print("연령대_세분화 결측치 수:", df['연령대_세분화'].isnull().sum())
print("연령대_세분화 고유값 목록:", df['연령대_세분화'].dropna().unique())

연령대_단순 결측치 수: 0
연령대_단순 고유값 목록: ['30~50대' '20대' '10대' '60대 이상']
연령대_세분화 결측치 수: 0
연령대_세분화 고유값 목록: ['40대' '30대' '20대' '50대' '10대' '60대 이상']


- ANOVA 검정 안정 확보를 위해 로그 변환 변수 추가

In [59]:
df['log_총액'] = np.log(df['총액1인TOT_개별국제교통비제외2'])

### 24년도 전처리 데이터 저장

In [60]:
df['year'] = 2024  # 연도 컬럼 붙이기
df.to_csv("data/2024/외래관광객조사_2024_전처리3(세분화).csv", index=False, encoding='utf-8-sig')